In [ ]:
import matplotlib.pyplot as plt
from Genetic.Representation import Function
from Genetic.NeuroEvolution import NeuroEvolution
import tensorflow.keras as keras
from tensorflow.keras import layers
from tensorflow.keras.losses import mse
import tensorflow as tf
import tensorflow.keras.backend as K
import pandas as pd
import seaborn as sns
from IPython.display import display
import numpy as np
from Model.model import SurvModel, FlchainModel, NwtcoModel, SupportModel
from Data.dataset import Flchain, Support, Nwtco
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from scipy.stats import f_oneway, ttest_rel

from Utils.utils import Experiment, FunctionHelper, StatsHelper
from itertools import combinations

In [ ]:
surv_reg1_func = [1, 9, 9, 1, 5, 12, 7, 9, 7]
surv_reg2_func = [1, 9, 12, 1, 5, 12, 7, 9, 7]

surv_reg1_f = FunctionHelper(surv_reg1_func)
surv_reg2_f = FunctionHelper(surv_reg2_func)

surv_reg1_f.plot_surv_loss()
surv_reg2_f.plot_surv_loss()

In [ ]:
model_class = SupportModel
ds = Support('Data/support2.csv')

In [ ]:
(x_train, ye_train, y_train, e_train,
 x_val, ye_val, y_val, e_val,
 x_test, ye_test, y_test, e_test) = ds.get_train_val_test_from_splits(test_id=0, val_id=1)
xy = (x_train, ye_train), (x_val, ye_val), (x_test, ye_test)

In [ ]:
exp_fix_left = [    
    [
    [4, 24, 26, 1, 5, 8, 10, 18, 26],
    [0.8772409, 0.86199677, 0.87765175, 0.8729801, 0.86985534]
    ]
    ,
    [
    [1, 13, 21, 6, 1, 17, 16, 26, 23],
    [0.8761697, 0.86079067, 0.8799163, 0.87499064, 0.8691531]
    ]
    ,
    [
    [1, 30, 17, 2, 2, 12, 7, 27, 21],
    [0.87695175, 0.8607952, 0.8758574, 0.8746895, 0.86952513]
    ]
    ,
    [
    [5, 16, 26, 2, 1, 23, 21, 26, 13],
    [0.8772601, 0.8598947, 0.8762512, 0.8748148, 0.8704117]
    ]
    ,
    [
    [3, 19, 24, 4, 2, 9, 7, 26, 22],
    [0.8768213, 0.8613667, 0.87783444, 0.8758533, 0.8684299]
    ]
    ,
    [
    [2, 25, 19, 1, 2, 22, 11, 27, 20],
    [0.87629455, 0.85960054, 0.87423044, 0.8746452, 0.87202144]
    ]
    ,
    [
    [4, 29, 23, 1, 5, 28, 10, 12, 26],
    [0.87911654, 0.8607826, 0.8786061, 0.8766557, 0.87129545]
    ]
    ,
    [
    [6, 19, 28, 3, 2, 27, 18, 25, 26],
    [0.87640166, 0.8616734, 0.8762457, 0.87446105, 0.8708326]
    ]
    ,
    [
    [6, 20, 26, 5, 6, 15, 15, 26, 8],
    [0.8772409, 0.86199677, 0.87765175, 0.8729801, 0.86985534]
    ]
    ,
    [
    [4, 16, 17, 5, 5, 11, 14, 29, 26],
    [0.87617105, 0.8618076, 0.8731987, 0.8757993, 0.8702609]
    ]
]


exp_fix_right = [
    [
    [2, 12, 18, 5, 5, 10, 21, 30, 13],
    [0.87411094, 0.8588948, 0.87764794, 0.8668676, 0.86477727]
    ]
    ,
    [
    [2, 12, 30, 3, 5, 18, 11, 17, 8],
    [0.87414896, 0.85872793, 0.8756539, 0.8677102, 0.8669584]
    ]
    ,
    [
    [4, 24, 22, 2, 5, 11, 19, 9, 21],
    [0.86574346, 0.85022885, 0.863207, 0.85962385, 0.8488512]
    ]
    ,
    [
    [2, 17, 18, 5, 4, 28, 23, 27, 26],
    [0.87232566, 0.8537298, 0.8730966, 0.8613159, 0.860814]
    ]
    ,
    [
    [1, 28, 19, 2, 3, 11, 17, 7, 20],
    [0.87635666, 0.86017835, 0.8806345, 0.8711718, 0.870226]
    ]
    ,
    [
    [4, 29, 25, 2, 6, 13, 30, 20, 14],
    [0.8657668, 0.84920144, 0.8708314, 0.8643102, 0.8586067]
    ]
    ,
    [
    [1, 12, 11, 5, 5, 11, 30, 23, 21],
    [0.8757766, 0.8564313, 0.8773052, 0.8652497, 0.8636394]
    ]
    ,
    [
    [4, 20, 7, 2, 3, 13, 17, 8, 7],
    [0.866109, 0.84694463, 0.8608435, 0.8594902, 0.8521]
    ]
    ,
    [
    [3, 11, 25, 2, 6, 29, 20, 14, 25],
    [0.86353123, 0.84722203, 0.86175334, 0.8571675, 0.8485608]
    ]
    ,
    [
    [4, 10, 12, 6, 3, 15, 16, 15, 30],
    [0.8768931, 0.8579675, 0.8747246, 0.8729199, 0.869685]
    ]
]


exp_opt_both = [
    [
    [5, 16, 25, 4, 3, 27, 10, 22, 12],
    [0.8672793, 0.84597194, 0.8634279, 0.8487187, 0.84988296]
    ]
    ,
    [
    [5, 22, 28, 6, 6, 17, 16, 22, 15],
    [0.8764243, 0.86214036, 0.8789259, 0.87793785, 0.87365144]
    ]
    ,
    [
    [4, 29, 17, 5, 1, 9, 10, 24, 29],
    [0.8819946, 0.8634803, 0.882649, 0.87231207, 0.87245286]
    ]
    ,
    [
    [3, 20, 18, 2, 6, 11, 28, 17, 16],
    [0.8812691, 0.8650252, 0.88095015, 0.8795607, 0.8745582]
    ]
    ,
    [
    [2, 10, 13, 2, 3, 16, 7, 26, 17],
    [0.87773484, 0.8610649, 0.8783397, 0.875197, 0.86913913]
    ]
    ,
    [
    [5, 17, 25, 5, 2, 11, 21, 26, 16],
    [0.88046366, 0.86280876, 0.8812849, 0.8716575, 0.87339664]
    ]
    ,
    [
    [2, 11, 24, 5, 2, 30, 24, 25, 8],
    [0.8798274, 0.8637887, 0.8800903, 0.8772788, 0.87508523]
    ]
    ,
    [
    [5, 14, 17, 3, 5, 8, 12, 18, 20],
    [0.8777477, 0.8639257, 0.8754931, 0.8790705, 0.87415475]
    ]
    ,
    [
    [1, 11, 23, 2, 3, 25, 10, 22, 15],
    [0.8809953, 0.8639982, 0.882283, 0.87423015, 0.87411565]
    ]
    ,
    [
    [1, 18, 26, 5, 1, 28, 23, 12, 15],
    [0.87382, 0.858814, 0.8776472, 0.87495846, 0.8695995]
    ]

]

# 5 Folds CV

In [ ]:
for i in range(10):
    for j in range(10):
        func_l, folds5_l = exp_fix_left[i]
        func_r, folds5_r = exp_fix_right[j]
        StatsHelper.plot_box(folds1=folds5_l, name1=f'fix_left_{i}', folds2=folds5_r, name2=f'fix_right_{j}')
    print('--------------------------------------')
        